Training and forecasting the solar power of Hokkaido Island in Japna
Training 2017, Forecast 2018

https://stackoverflow.com/questions/35139108/how-to-install-xgboost-in-anaconda-python-windows-platform

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import csv 
import matplotlib.pyplot as plt
import pytz
import xgboost as xgb

In [2]:
import os
cwd = os.getcwd()
cwd
# os.chdir('pvinhokkaidodatadistribution')
# C:\Users\Mhdella\Google Drive\Jupyter Folder\PV_Hokkaido_forecast_contest\
os.chdir('C:\\Users\\Mhdella\\Google Drive\\Jupyter Folder\\PV_Hokkaido_forecast_contest\\pvinhokkaidodatadistribution')
nwd=os.getcwd()
nwd

'C:\\Users\\Mhdella\\Google Drive\\Jupyter Folder\\PV_Hokkaido_forecast_contest\\pvinhokkaidodatadistribution'

In [3]:
# file_name='Merg_Ens_Agg_42_44_mes17_18.csv'
# file_name='Merg_Ens_Agg_43_mes17_18.csv'

# file_name='Merg_Ens_Agg_42_44_mes16_17_18.csv'
file_name='Merg_Ens_mmaxmin_Agg_42_44_mes16_18.csv'

# file_name='Merg_Res_Comb_S1_S2_2018_tz_Jp.csv'
# file_mane='Merg_Res_Comb_S1_S2_2017_tz_Jp.csv'

df_hok=pd.read_csv(file_name) 
df_hok.head()
# df_hok.tail()

,DATE,TIME,S1[MW],S2[MW],solar[kW/m2],temp[deg C],solar[kW/m2].1,temp[deg C].1,solar[kW/m2].2,temp[deg C].2,...,t2m.2,asn.2,sd.2,ssrdc.2,ssrd.2,tcc.2,tciw.2,tclw.2,tp.2,wind_speed.2
0,1/1/2016,9:00:00,54.7,78.4,0.159228,-1.2,0.276627,0.0,0.339371,-5.9,...,-13.909399,0.600965,7.450000e-09,57.319398,30.413241,0.000000,0.000000,0.000000,0.000000e+00,7.704484
1,1/1/2016,9:30:00,113.8,89.0,0.221269,-0.3,0.307764,0.1,0.297246,-4.4,...,-12.694835,0.600870,7.450000e-09,104.414120,55.211142,0.037510,0.000396,0.000012,2.690000e-07,7.772519
2,1/1/2016,10:00:00,115.9,97.3,0.231619,-0.5,0.354492,0.7,0.320564,-2.3,...,-11.480271,0.600775,7.450000e-09,151.508843,80.009043,0.075021,0.000793,0.000023,5.390000e-07,7.840554
3,1/1/2016,10:30:00,139.5,100.6,0.205909,-0.4,0.363168,0.8,0.306980,-1.9,...,-10.265707,0.600680,7.450000e-09,198.603565,104.806944,0.112531,0.001189,0.000034,8.080000e-07,7.908589
4,1/1/2016,11:00:00,125.7,96.8,0.277444,-0.2,0.358989,0.6,0.237579,-3.1,...,-9.051143,0.600586,7.450000e-09,245.698287,129.604846,0.150041,0.001585,0.000046,1.080000e-06,7.976625


In [4]:
#### This cell just to check out the dataset with different coordination lat 43 and lat42-44 from ECMWF
df_hok1=pd.read_csv('Merg_Ens_Agg_42_44_mes17_18.csv') 
df_hok2=pd.read_csv('Merg_Ens_Agg_43_mes17_18.csv')

df_mns= {'col1': df_hok1.mean(), 'col2': df_hok2.mean()}
# df_mns

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html

https://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_regression.html#sphx-glr-auto-examples-ensemble-plot-gradient-boosting-regression-py

In [5]:
import sklearn 
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_friedman1
# X, y = make_friedman1(n_samples=1200, random_state=0, noise=1.0)
# X, y = make_friedman1(n_samples=1200, random_state=0, noise=1.0)
df_hok.head()

,DATE,TIME,S1[MW],S2[MW],solar[kW/m2],temp[deg C],solar[kW/m2].1,temp[deg C].1,solar[kW/m2].2,temp[deg C].2,...,t2m.2,asn.2,sd.2,ssrdc.2,ssrd.2,tcc.2,tciw.2,tclw.2,tp.2,wind_speed.2
0,1/1/2016,9:00:00,54.7,78.4,0.159228,-1.2,0.276627,0.0,0.339371,-5.9,...,-13.909399,0.600965,7.450000e-09,57.319398,30.413241,0.000000,0.000000,0.000000,0.000000e+00,7.704484
1,1/1/2016,9:30:00,113.8,89.0,0.221269,-0.3,0.307764,0.1,0.297246,-4.4,...,-12.694835,0.600870,7.450000e-09,104.414120,55.211142,0.037510,0.000396,0.000012,2.690000e-07,7.772519
2,1/1/2016,10:00:00,115.9,97.3,0.231619,-0.5,0.354492,0.7,0.320564,-2.3,...,-11.480271,0.600775,7.450000e-09,151.508843,80.009043,0.075021,0.000793,0.000023,5.390000e-07,7.840554
3,1/1/2016,10:30:00,139.5,100.6,0.205909,-0.4,0.363168,0.8,0.306980,-1.9,...,-10.265707,0.600680,7.450000e-09,198.603565,104.806944,0.112531,0.001189,0.000034,8.080000e-07,7.908589
4,1/1/2016,11:00:00,125.7,96.8,0.277444,-0.2,0.358989,0.6,0.237579,-3.1,...,-9.051143,0.600586,7.450000e-09,245.698287,129.604846,0.150041,0.001585,0.000046,1.080000e-06,7.976625


In [6]:
print(list(df_hok))

['DATE', 'TIME', 'S1[MW]', 'S2[MW]', 'solar[kW/m2]', 'temp[deg C]', 'solar[kW/m2].1', 'temp[deg C].1', 'solar[kW/m2].2', 'temp[deg C].2', 'solar[kW/m2].3', 'temp[deg C].3', 'u10', 'v10', 't2m', 'asn', 'sd', 'ssrdc', 'ssrd', 'tcc', 'tciw', 'tclw', 'tp', 'wind_speed', 'u10.1', 'v10.1', 't2m.1', 'asn.1', 'sd.1', 'ssrdc.1', 'ssrd.1', 'tcc.1', 'tciw.1', 'tclw.1', 'tp.1', 'wind_speed.1', 'u10.2', 'v10.2', 't2m.2', 'asn.2', 'sd.2', 'ssrdc.2', 'ssrd.2', 'tcc.2', 'tciw.2', 'tclw.2', 'tp.2', 'wind_speed.2']


In [7]:
#### df_tm = df_hok[df_hok['ssrdc'] != 0] ## dropping night hours, when clear-sky solar irradiance (SSRDC) forecasts=0, and this can be applied in real forecasts  

#### df_tm=df_tm.set_index('DATE')

############################# Not dropping night hours

df_tm=df_hok.set_index('DATE') 

############## In case of forecast solar power  #######

Targ=df_tm['S1[MW]']+df_tm['S2[MW]']
Targ = Targ.replace(np.nan, 0, regex=True)
df_tm=df_tm[['u10', 'v10', 't2m', 'asn', 'sd', 'ssrdc', 'ssrd', 'tcc', 'tciw', 'tclw', 'tp', 'wind_speed', 
             'u10.1', 'v10.1', 't2m.1', 'asn.1', 'sd.1', 'ssrdc.1', 'ssrd.1', 'tcc.1', 'tciw.1', 'tclw.1', 'tp.1', 'wind_speed.1', 
             'u10.2', 'v10.2', 't2m.2', 'asn.2', 'sd.2', 'ssrdc.2', 'ssrd.2', 'tcc.2', 'tciw.2', 'tclw.2', 'tp.2', 'wind_speed.2']]

############### In case of forecast solar irradiance  #######

# Targ=df_tm['ssrd']
# df_tm=df_tm[['u10', 'v10', 't2m', 'asn', 'sd', 'ssrdc', 'tcc', 'tciw', 'tclw', 'tp', 'wind_speed', 
#              'u10.1', 'v10.1', 't2m.1', 'asn.1', 'sd.1', 'ssrdc.1', 'tcc.1', 'tciw.1', 'tclw.1', 'tp.1', 'wind_speed.1', 
#              'u10.2', 'v10.2', 't2m.2', 'asn.2', 'sd.2', 'ssrdc.2', 'tcc.2', 'tciw.2', 'tclw.2', 'tp.2', 'wind_speed.2']]

df_tm.head()
df_tm.tail()

# df_tm['Date']=df_hok['DATE']

Targ.tail()


DATE
12/31/2018    0.0
12/31/2018    0.0
12/31/2018    0.0
12/31/2018    0.0
12/31/2018    0.0
dtype: float64

In [8]:
len(df_tm)
# len(Targ)
print( len(Targ), max(Targ), Targ.mean())

38961 312.6 50.475909242576236


In [9]:
X_train=df_tm.loc['1/1/2016':'12/31/2016']
X_test=df_tm.loc['1/1/2017':'12/31/2017']

# X_train=df_tm.loc['1/1/2017':'12/31/2017']
# X_test=df_tm.loc['1/1/2018':'12/31/2018']

Y_train=Targ.loc['1/1/2016':'12/31/2016']
Y_test=Targ.loc['1/1/2017':'12/31/2017']

# Y_train=Targ.loc['1/1/2017':'12/31/2017']
# Y_test=Targ.loc['1/1/2018':'12/31/2018']

X_train.head()
X_train.tail()
len(X_train)

# Y_train.head()
# len(Y_train)
Y_train.tail()

DATE
12/31/2016    0.0
12/31/2016    0.0
12/31/2016    0.0
12/31/2016    0.0
12/31/2016    0.0
dtype: float64

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

est = GradientBoostingRegressor(n_estimators=100, learning_rate=0.05,
                                max_depth=8, random_state=0, subsample=0.9, loss='ls').fit(X_train, Y_train)
pred_Y=est.predict(X_test)
mse=mean_squared_error(Y_test, pred_Y)  
nRMSE=np.sqrt(mse)/Targ.mean()

print(mse, nRMSE)

Here including mean, max and min of the ensembles of weather forecasts

Parameters: n_estimators=400, learning_rate=0.1, max_depth=4, random_state=0, loss='ls'

with training set of 2017 only, (Solar irr, nRMSE=0.1872461)

with training set of 2016+2017, (Solar irr, nRMSE=0.1770770)

For Solar Power: 2016 train and 2017 test

nRMSE=1.2924510, ### Improvement by case of using mean, max, min of ensemble over using just mean, with samilar GB model's params, improv=(1.44191-1.2924)/1.44191*100=10.3688%

Parameters: n_estimators=800, learning_rate=0.1, max_depth=4, random_state=0, loss='ls', nRMSE=1.30852099

Parameters: n_estimators=400, learning_rate=0.1, max_depth=8, random_state=0, loss='ls', nRMSE=1.27297 <1.2924@depth=4



np.sqrt(mse)/max(Targ)=0.15224 @nRMSE=1.2737 

In [ ]:
# import xgboost as xgb
# model.fit(X_train, Y_train)

from xgboost.sklearn import XGBRegressor

model=XGBRegressor(n_estimators=200, learning_rate=0.05, max_depth=8, random_state=0,
                 subsample=0.9, colsample_bytree=1.0, loss='ls').fit(X_train, Y_train)


model.score(X_test,Y_test)

pred_Yxgb=model.predict(X_test)
mse=mean_squared_error(Y_test, pred_Yxgb)  
nRMSE=np.sqrt(mse)/Targ.mean()
# nRMSE=np.sqrt(mse)/max(Targ)

print(mse, nRMSE)

In [ ]:
#print(mse, nRMSE)

By XGB, nRMSE=1.3115674 > 1.2729759 from GB, but XGBoost here without any tuning for its parameters

Cross-validation for GBoosting: 2folds for each of 100 candidates, totalling 200 fits (each fold 1 year)

In [ ]:
from sklearn import metrics   ##Additional scklearn functions
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV   #Perforing grid search
from sklearn.ensemble import GradientBoostingRegressor
GB_model = GradientBoostingRegressor()

Xgrid_train=df_tm.loc['1/1/2016':'12/31/2017']
Ygrid_train=Targ.loc['1/1/2016':'12/31/2017']

test_params = {'n_estimators':[100, 200, 300, 400, 500], 'learning_rate':[0.05,0.1, 0.2, 0.3, 0.5], 
               'max_depth':[2,4,8,12]}

# grid = GridSearchCV(estimator = GB_model, param_grid = test_params, n_jobs=4, 
#                     scoring='neg_mean_squared_error',cv=3, verbose=3)

grid = GridSearchCV(estimator = GB_model, param_grid = test_params, n_jobs=4, 
                    scoring='neg_mean_squared_error',cv=2, verbose=3)

grid.fit(Xgrid_train,Ygrid_train)
print(grid.best_params_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   47.9s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed: 21.5min
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed: 49.6min
[Parallel(n_jobs=4)]: Done 300 out of 300 | elapsed: 56.0min finished
{'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 100}  Tried GB with these parms but got 1.295197 > 1.2729759 (GB without CV, Why? Because CV is with 3 folds among the training data, while without CV is for all trainin dataset at once. CV's parmeters is more reliable to use for testing for real solar power forecasts in 2018)

CV GB with 2 folds each a year, {'learning_rate': 0.05, 'max_depth': 8, 'n_estimators': 100}, nRMSE=1.258419 (Same nRMSE of CV with less than a year, by another script) 

Cross-validation for XGBoost: 2folds for each 400 candidates (total 1200 fits), takes almost 2hours

In [10]:
from sklearn import metrics   ##Additional scklearn functions
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV   #Perforing grid search
from xgboost.sklearn import XGBRegressor
xgb_model = XGBRegressor()

Xgrid_train=df_tm.loc['1/1/2016':'12/31/2017']
Ygrid_train=Targ.loc['1/1/2016':'12/31/2017']

# test_params = {'n_estimators':[100, 200, 300, 400, 500], 'learning_rate':[0.05,0.1, 0.2, 0.3, 0.5], 
#                'max_depth':[2,4,8,12],'subsample': [0.9, 1.0], 'colsample_bytree': [0.9, 1.0]}

test_params = {'n_estimators':[100, 200, 300, 500], 'learning_rate':[0.05,0.1, 0.2], 
               'max_depth':[2,4,8,10]}

grid = GridSearchCV(estimator = xgb_model, param_grid = test_params, n_jobs=4, 
                    scoring='neg_mean_squared_error',cv=2, verbose=3)

grid.fit(Xgrid_train,Ygrid_train)
print(grid.best_params_)

Fitting 2 folds for each of 48 candidates, totalling 96 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:  1.3min
[Parallel(n_jobs=4)]: Done  96 out of  96 | elapsed:  7.3min finished


{'learning_rate': 0.05, 'max_depth': 8, 'n_estimators': 200}


Fitting 3 folds for each of 400 candidates, totalling 1200 fits

{'colsample_bytree': 1.0, 'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 100, 'subsample': 0.9}

With best CV XGB, nRMSE=1.295126 > 1.2729759 from GB, (nRMSE was XGBoost, nRMSE=1.3115674 before CV)

CV XGBoost with 2 folds each a year, {'learning_rate': 0.05, 'max_depth': 8, 'n_estimators': 200}, nRMSE=1.257295 (Same nRMSE of CV with less than a year, by another script). 

Note: nRMSE=1.239930 by dding (subsample=0.9, colsample_bytree=1.0), which is less than < 1.257295(in case of no sample options are used). However, improv=(1.257-1.239)/1.257=1.38% (not significant)

If nRMSE normalized with respect to max(Targ) insead of Mean(Targ), it would be=1.239*Targ.mean()/max(Targ)=0.148, still higher comaring to solar forecasts in Australia, (the weather is more challenging in Hokkaido, JP), SInce the variability normalized solar power of daily hours is (np.std(Targ[Targ>0]/max(Targ))=0.29649). Thus, the variability of Sol Pw 0.30(at Hokkaido) > 0.25(at Australia case), Sol Pw in Hokkaido is about variable by 1.2 times than in Australia.


In [ ]:
##### Printing out the CV results
print("Best: %f using %s" % (grid.best_score_, grid.best_params_))

means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Note: By dropping or filtering the night hours, nRMSE=1.05639 (XGB) and (1.058 by GB). So removing the night hour improve the performance significantly, (1.2399-1.05639)/1.2399)*100=14.8

By using the condition when clear-sky irradiance equals to zero to filter the night hours from the data, the nRMSE=1.0694 < 1.23, improves by 13.8% and it is just 1.2% higher of 1.05 which using actual power to filter the night hours. Using clear-sky is more practical than using actual solar power to filter the nigth hours, because actual power is not seen in the forecasting period.

1.2399*Targ.mean()/max(Targ) ### 1.2399 in case of Targ with all hours (day+nigh)
It gives 0.148

In case of all hours:

print( len(Targ), max(Targ), Targ.mean())

52603,  312.6,  37.39

Whereas, in case of keeping daily hours only and filtering the night hours:
print( len(Targ), max(Targ), Targ.mean())

38961,  312.6,  50.48

So, here is a question, the nRMSE as teh contest's selection criteria, the hours of calculating the mean actual power in the denomerator; Q:Are these hours day hours only or all hours (including the night hours)?

This is a critical question, because excluding night hours incerease the mean of actual solar power and thus  decrease nRMSE. As it happened all hours nRMSE= 1.23, no night horus nRMSE=1.07. While the rmse of all hours is less than RMSE of day hours only (0.147<0.1727)

# Note
Therefore, in fact, filtering the night hours does not imporve the forecast but it improve the nRMSE because the mean of actual solar power in the denomerator of RMSE becomes higher than the mean of actual solar power in case of including all hours. 
Let's check out the forecasts by from xgboost in both cases:(showing both mse and nRMSE)

1)All hours, including night hours in the data 
print(mse, nRMSE)
2148.8480,  1.2399

2)Without night hours in thd data
print(mse, nRMSE)
2914.0024, 1.0694

Thus, all hours forecasts is better because they have less MSE. MSE is considered for comparison since it does not include a division by mean of actual solar power as in nRMSE, which can be misleading when comparing between the two cases, with and without night hours.

But the CV for tuning parameters have data of all hours, while the CV has not conducted for without night hours case yet. So the parameters are more suitable for all hours case and hence leads to better forecasts. 

CV search without night hours data has been conducted and led to the same parameters that CV gave with all hours data: {'learning_rate': 0.05, 'max_depth': 8, 'n_estimators': 200}
 
Thus, now it is obvious that all hours case gives better forecasts.

In [ ]:
1.07*50.48/ 312.6

In [ ]:
1.23*37.39/ 312.6

In [ ]:
np.std(Targ[Targ>0]/max(Targ))

In [ ]:
(1.2399-1.05639)/1.2399*100

In [ ]:
(1.2399-1.0694)/1.2399*100